In [4]:
# https://docs.cohere.com/reference/generate

import cohere
import os

co = cohere.Client(os.environ['COHERE_API_KEY'])

prompt = """
From the following historical document text, please grab out the following items and return with the following format
[{{
  "name": "John Doe",
  "age": 30,
  "title": "Farmer"
}},
{{<ITEM 2>}}, {{Item 3>}}, ...]

Here is the text:
{text}"""


response = co.generate(
  prompt='From the following',
)
print(response)


SyntaxError: incomplete input (3923542006.py, line 8)

In [72]:
# pip install -U langchain-community cohere
## Use Langchain https://python.langchain.com/docs/integrations/llms/cohere
# https://api.python.langchain.com/en/latest/llms/langchain_community.llms.cohere.Cohere.html

#from langchain_community.llms import Cohere
# from langchain_core.prompts import PromptTemplate

from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import json

model = OpenAI()

system_message = """
From the following historical document text, please grab out the following items and return with the following format
[{{
  "name": "John Doe",
  "age": 30,
  "title": "Farmer"
}},
{{<ITEM 2>}}, {{Item 3>}}, ...]

Return only the list of Python dictionary objects and nothing more. Make a dictionary entry for each person mentioned.

Here is the text:
{text}"""

text = "Tim Paulson was a farmer who lived in the 1800s. He was 30 years old and lived in the state of California. He had two kids, John and Jane. John was 12 and Jane was 10. They were both students."

prompt = PromptTemplate(input_variables=['text'], template=system_message)
full_prompt = prompt.format(text=text)

response_raw = model.predict(full_prompt)

response = json.loads(response_raw)

response




[{'name': 'Tim Paulson', 'age': 30, 'title': 'Farmer'},
 {'name': 'John Paulson', 'age': 12, 'title': 'Student'},
 {'name': 'Jane Paulson', 'age': 10, 'title': 'Student'}]

In [33]:
# pip install -U langchain-community cohere
## Use Langchain https://python.langchain.com/docs/integrations/llms/cohere
# https://api.python.langchain.com/en/latest/llms/langchain_community.llms.cohere.Cohere.html

from langchain_community.llms import Cohere
from langchain_core.prompts import PromptTemplate

# from langchain_openai import OpenAI
# from langchain.prompts import PromptTemplate

import json

model = Cohere()

system_message = """
From the following historical document text, please grab out the following items and return with the following format
[{{
  "name": "John Doe",
  "age": 30,
  "title": "Farmer"
}},
{{<ITEM 2>}}, {{Item 3>}}, ...]

Return only the list of Python dictionary objects and nothing more. Make a dictionary entry for each person mentioned.

Here is the text:
{text}

NO EXTRA TEXT BUT THE LIST!!!!"""

text = "Tim Paulson was a farmer who lived in the 1800s. He was 30 years old and lived in the state of California. He had two kids, John and Jane. John was 12 and Jane was 10. They were both students."

prompt = PromptTemplate(input_variables=['text'], template=system_message)
full_prompt = prompt.format(text=text)

response_raw = model.predict(full_prompt)

# response = json.loads(response_raw)

# Regex get the text within ```json<TEXT>``` and return it as a list of dictionaries
import re
pattern = r"```json(.*)```"
match = re.findall(pattern, response_raw, re.MULTILINE | re.DOTALL)[0]

response = json.loads(match)

response








[{'name': 'Tim Paulson', 'age': 30, 'title': 'Farmer'},
 {'name': 'John', 'age': 12, 'title': 'Student'},
 {'name': 'Jane', 'age': 10, 'title': 'Student'}]

## Process contracts.csv

In [132]:
import pandas as pd

base_directory = './input'

df = pd.read_csv(f"{base_directory}/contract-records.csv")
df = df.replace("_x000D_", "", regex=True)

# remap values that have Apprenticeship Agreement to Apprenticeship Agreements
df['sub_category'] = df['sub_category'].replace('Apprenticeship Agreements', 'Apprenticeship Agreement')

df['sub_category'].value_counts()

apprenticeship_agreements = df[df['sub_category'] == 'Apprenticeship Agreement']


In [133]:
import ast

def from_same_document(document1, document2):
    system_message = """
    Your job is to determine whether the following two documents are from the same document or not. Please return a boolean value of True or False. 
    One of the key things to look out for is the same names, locations, or dates referred to in both documents.

    Document 1:
    {document1}

    Document 2:
    {document2}

    ---
    IMPORTANT: Make sure to only return the boolean value and nothing else
    """
    
    # Fail fast if the documents are not from the same project
    if document1['project_id'] != document2['project_id']:
        return False
    
    prompt = PromptTemplate(input_variables=['document1', 'document2'], template=system_message)
    full_prompt = prompt.format(document1=document1['transcription_text'], document2=document2['transcription_text'])

    response_raw = model.predict(full_prompt)

    print(response_raw.strip())

    return ast.literal_eval(response_raw.strip())


In [134]:
apprenticeship_agreements.head()

,project_id,category,sub_category,transcription_text,document_url
0,11406,Contracts,Apprenticeship Agreement,Copy\n\nAgreement of Apprenticeship\nBu \nMrs. Kate V.Chamblin \nof the first part and\n\n1st Lt. Geo. W. Rollins\nV.R.C. U.S. Army Agt\nfor Carroll Parish La. of \nthe second part\n\nMinor Orphans Freed\nJames Samenett\nBetty Taylor\nMary Taylor \n\nLake Providence\nCarroll Parish La\nDated Oct. 1 1866\nExec'd Nov. 3 1866\n,https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00439
1,11406,Contracts,Apprenticeship Agreement,"[H 132 ENCL] \n\nCopy\n\nAgreement of Apprenticeship.\n\nThis agreement in two parts made & entered into this 1"" day of October A. D. 1866 by & between Mrs Kate V. Chambliss of the first part and 1st Lieut Geo. W. Rollins Vet Res. Corps U.S.A. Agent for Carroll Parish La. Bur of Ref. Freedn & Aband Lands La and by virtue of authority contained in Circular No 25 dated Hd. Qrs. Bur Ref Freedn & Abd Lands New Orleans Louisiana Octr 31"" 1865 Guardian for Minors & orphans of Freedmen for Carroll Parish La party hereto of the second part. Witnesseth That James Samenett, Betty Taylor and Mary Taylor minor orphans of African decent are hereby bound & apprenticed to service to the said Mrs Kate V. Chambliss party of the first part & undersigned during their years of minority commencing & ending as follows: James Samenett aged 14 years, commencing on the 1"" day of Oct. A. D. 1866 & terminating on the 1"" day of October 1870. Betty Taylor aged 9 years commencing on the 1"" day of Oct. A. D. 1866 & ending on the 1"" day of Oct. A. D. 1872. Mary Taylor aged 7 years commencing on the 1"" day of October A. D. 1866 & terminating on the 1"" day of October A. D. 1874. And it",https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00440
2,11406,Contracts,Apprenticeship Agreement,"is agreed on the part of the party of the first part & undersigned Mrs Kate V. Chambliss that for the consideration of the faithfull services to be rendered by the within named & said minor orphans the said minor orphans shall receive comfortable clothing, board medical treatment when sick, a reasonable amount of schooling and permission to attend church each Sabbath and at the end or termination of their term of apprenticeship the said minor orphans shall be allowed to retain all articles of their personal apparel.\n \nThe said parties do hereby mutually agree that all laws or parts of laws enacted or that may be enacted by the United States or the State of Louisiana establishing laws for the welfare or government of Freedmens minors or orphans or for the government of the same under laws establishing the Bur of Ref Freed & Ab^d Lands or in any way affecting the provisions of this agreement is & shall be made a part of this agreement and that all laws applying to white minors or orphans shall be extended to the said & within named minor orphans during the conti-",https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00441
3,11406,Contracts,Apprenticeship Agreement,"continuance of this agreement & finally that this agreement shall expire & terminate when the aforesaid minor male orphan shall have arrived at the age of eighteen years and the minor female orphans shall have arrived at the age of fifteen years respectively.\n \nIn testimony whereof the said parties have hereunto affixed their names to this agreement. Done at Lake Providence Louisiana Parish of Carroll on the third day of November A. D. 1866.\n\nsigned Kate V. Chambliss \nsigned Geo. W Rollins\n1st Lieut. VRC. U.S. Army\nAgent for Carrol Parish \nLa. Bureau of R. F. & A Lands\nLouisiana\n\nExecuted in Presence of\nSigned Ben C. Johnson\nJackson Chambliss\nJohn A. Ginst [[?]]\n\n[[image - three boxes oriented vertically representing Internal Revenue Stamps that appeared on original document, reading as follows: 2� / Int Rev / [[illegible]]; a single illegible initial appears beneath each ""stamp""]]\n\nI certify the above to be a true copy of the ori

In [135]:
merge_id = 0
for i in range(0, len(apprenticeship_agreements)):
    # Access the row index instead of the row itself
    index1 = apprenticeship_agreements.index[i]
    index2 = apprenticeship_agreements.index[i+1] if i + 1 < len(apprenticeship_agreements) else None

    # Use loc to access the rows
    document1 = apprenticeship_agreements.loc[index1]
    document2 = apprenticeship_agreements.loc[index2] if index2 is not None else None

    if document2 is not None:
        if from_same_document(document1, document2):
            # Set the value using loc
            apprenticeship_agreements.loc[index1, 'merge_id'] = merge_id
            apprenticeship_agreements.loc[index2, 'merge_id'] = merge_id
            continue
        
    apprenticeship_agreements.loc[index1, 'merge_id'] = merge_id
    merge_id += 1

    if i > 15:
        break


True


/var/folders/rz/zcgcqm0x1bl9cj8slq9l2s1c0000gn/T/ipykernel_20895/2402642848.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apprenticeship_agreements.loc[index1, 'merge_id'] = merge_id


True
True
False
True
False
True
False
False
True
False
True
False


In [136]:
# Show more of the transcription_text area by modifying pd settings
pd.set_option('display.max_colwidth', 2000)

apprenticeship_agreements.head(10)


,project_id,category,sub_category,transcription_text,document_url,merge_id
0,11406,Contracts,Apprenticeship Agreement,Copy\n\nAgreement of Apprenticeship\nBu \nMrs. Kate V.Chamblin \nof the first part and\n\n1st Lt. Geo. W. Rollins\nV.R.C. U.S. Army Agt\nfor Carroll Parish La. of \nthe second part\n\nMinor Orphans Freed\nJames Samenett\nBetty Taylor\nMary Taylor \n\nLake Providence\nCarroll Parish La\nDated Oct. 1 1866\nExec'd Nov. 3 1866\n,https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00439,0.0
1,11406,Contracts,Apprenticeship Agreement,"[H 132 ENCL] \n\nCopy\n\nAgreement of Apprenticeship.\n\nThis agreement in two parts made & entered into this 1"" day of October A. D. 1866 by & between Mrs Kate V. Chambliss of the first part and 1st Lieut Geo. W. Rollins Vet Res. Corps U.S.A. Agent for Carroll Parish La. Bur of Ref. Freedn & Aband Lands La and by virtue of authority contained in Circular No 25 dated Hd. Qrs. Bur Ref Freedn & Abd Lands New Orleans Louisiana Octr 31"" 1865 Guardian for Minors & orphans of Freedmen for Carroll Parish La party hereto of the second part. Witnesseth That James Samenett, Betty Taylor and Mary Taylor minor orphans of African decent are hereby bound & apprenticed to service to the said Mrs Kate V. Chambliss party of the first part & undersigned during their years of minority commencing & ending as follows: James Samenett aged 14 years, commencing on the 1"" day of Oct. A. D. 1866 & terminating on the 1"" day of October 1870. Betty Taylor aged 9 years commencing on the 1"" day of Oct. A. D. 1866 & ending on the 1"" day of Oct. A. D. 1872. Mary Taylor aged 7 years commencing on the 1"" day of October A. D. 1866 & terminating on the 1"" day of October A. D. 1874. And it",https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00440,0.0
2,11406,Contracts,Apprenticeship Agreement,"is agreed on the part of the party of the first part & undersigned Mrs Kate V. Chambliss that for the consideration of the faithfull services to be rendered by the within named & said minor orphans the said minor orphans shall receive comfortable clothing, board medical treatment when sick, a reasonable amount of schooling and permission to attend church each Sabbath and at the end or termination of their term of apprenticeship the said minor orphans shall be allowed to retain all articles of their personal apparel.\n \nThe said parties do hereby mutually agree that all laws or parts of laws enacted or that may be enacted by the United States or the State of Louisiana establishing laws for the welfare or government of Freedmens minors or orphans or for the government of the same under laws establishing the Bur of Ref Freed & Ab^d Lands or in any way affecting the provisions of this agreement is & shall be made a part of this agreement and that all laws applying to white minors or orphans shall be extended to the said & within named minor orphans during the conti-",https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00441,0.0
3,11406,Contracts,Apprenticeship Agreement,"continuance of this agreement & finally that this agreement shall expire & terminate when the aforesaid minor male orphan shall have arrived at the age of eighteen years and the minor female orphans shall have arrived at the age of fifteen years respectively.\n \nIn testimony whereof the said parties have hereunto affixed their names to this agreement. Done at Lake Providence Louisiana Parish of Carroll on the third day of November A. D. 1866.\n\nsigned Kate V. Chambliss \nsigned Geo. W Rollins\n1st Lieut. VRC. U.S. Army\nAgent for Carrol Parish \nLa. Bureau of R. F. & A Lands\nLouisiana\n\nExecuted in Presence of\nSigned Ben C. Johnson\nJackson Chambliss\nJohn A. Ginst [[?]]\n\n[[image - three boxes oriented vertically representing Internal Revenue Stamps that appeared on original document, reading as follows: 2� / Int Rev / [[illegible]]; a single illegible initial appears beneath each ""stamp""]]\n\nI certify the above to be a

In [137]:
# Step 1 & 2: Group by 'merge_id' and concatenate 'transcription_text'
aggregated_text = apprenticeship_agreements.groupby('merge_id')['transcription_text'].agg('\n'.join).reset_index()

# Step 3: Drop duplicate 'merge_id' rows, keeping the first occurrence
apprenticeship_agreements = apprenticeship_agreements.drop_duplicates(subset='merge_id')

# Step 4: Merge the aggregated text back into the original DataFrame
apprenticeship_agreements = pd.merge(apprenticeship_agreements, aggregated_text, on='merge_id', how='left', suffixes=('', '_aggregated'))

# Now, 'transcription_text_aggregated' contains the concatenated text
# You might want to rename or drop the original 'transcription_text' column as needed

In [138]:
apprenticeship_agreements

,project_id,category,sub_category,transcription_text,document_url,merge_id,transcription_text_aggregated
0,11406,Contracts,Apprenticeship Agreement,Copy\n\nAgreement of Apprenticeship\nBu \nMrs. Kate V.Chamblin \nof the first part and\n\n1st Lt. Geo. W. Rollins\nV.R.C. U.S. Army Agt\nfor Carroll Parish La. of \nthe second part\n\nMinor Orphans Freed\nJames Samenett\nBetty Taylor\nMary Taylor \n\nLake Providence\nCarroll Parish La\nDated Oct. 1 1866\nExec'd Nov. 3 1866\n,https://transcription.si.edu/transcribe/11406/NMAAHC-004567395_00439,0.0,"Copy\n\nAgreement of Apprenticeship\nBu \nMrs. Kate V.Chamblin \nof the first part and\n\n1st Lt. Geo. W. Rollins\nV.R.C. U.S. Army Agt\nfor Carroll Parish La. of \nthe second part\n\nMinor Orphans Freed\nJames Samenett\nBetty Taylor\nMary Taylor \n\nLake Providence\nCarroll Parish La\nDated Oct. 1 1866\nExec'd Nov. 3 1866\n\n[H 132 ENCL] \n\nCopy\n\nAgreement of Apprenticeship.\n\nThis agreement in two parts made & entered into this 1"" day of October A. D. 1866 by & between Mrs Kate V. Chambliss of the first part and 1st Lieut Geo. W. Rollins Vet Res. Corps U.S.A. Agent for Carroll Parish La. Bur of Ref. Freedn & Aband Lands La and by virtue of authority contained in Circular No 25 dated Hd. Qrs. Bur Ref Freedn & Abd Lands New Orleans Louisiana Octr 31"" 1865 Guardian for Minors & orphans of Freedmen for Carroll Parish La party hereto of the second part. Witnesseth That James Samenett, Betty Taylor and Mary Taylor minor orphans of African decent are hereby bound & apprenticed to service to the said Mrs Kate V. Chambliss party of the first part & undersigned during their years of minority commencing & ending as follows: James Samenett aged 14 years, commencing on the 1"" day of Oct. A. D. 1866 & terminating on the 1"" day of October 1870. Betty Taylor aged 9 years commencing on the 1"" day of Oct. A. D. 1866 & ending on the 1"" day of Oct. A. D. 1872. Mary Taylor aged 7 years commencing on the 1"" day of October A. D. 1866 & terminating on the 1"" day of October A. D. 1874. And it\nis agreed on the part of the party of the first part & undersigned Mrs Kate V. Chambliss that for the consideration of the faithfull services to be rendered by the within named & said minor orphans the said minor orphans shall receive comfortable clothing, board medical treatment when sick, a reasonable amount of schooling and permission to attend church each Sabbath and at the end or termination of their term of apprenticeship the said minor orphans shall be allowed to retain all articles..."
1,15369,Contracts,Apprenticeship Agreement,"[[preprinted]]\nBureau of Refugees, Freedman and Abandoned Lands,\nHead-Quarters, Asst. Commissioner, State of North Carolina\nRaleigh, N.C., ^[[March 2""]] 186^[[6]] [[/preprinted]]\n\nJim Tew (Colored) having made statements that he and his brother Charley & Joe have been bound by one Richd Holmes of Sampson County to one Lewis Tew. - Know all by these that the said Richd Holmes has no authority for binding or apprenticing minors, and that all indentures that may have been made by him are null and void. the aforesaid Jim Tew and his brother Charley and Joe are free to engage in any contract or to work with such as may wish to hire them.\n\nAll officers of the Bureau, Capts of Police or others are requested to see that these men are protected in their rights.\n\nBy order of Col. Whittlesey, Asst. Commissioner\n\nLieut. and A. A. A. Genl.",https://transcription.si.edu/transcribe/15369/NMAAHC-004567415_00230,1.0,"[[preprinted]]\nBureau of Refugees, Freedman and Abandoned Lands,\nHead-Quarters, Asst. Commissioner, State of North Carolina\nRaleigh, N.C., ^[[March 2""]] 186^[[6]] [[/preprinted]]\n\nJim Tew (Colored) having made statements that he and his brother Charley & Joe have been bound by one Richd Holmes of Sampson County to one Lewis Tew. - Know all by these that the said Richd Holmes has no authority for binding or apprenticing minors, and that all indentures that may have